In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import zipfile

# Path ke file ZIP
zip_files = {
    "/content/drive/MyDrive/T5_Checkpoints/t5-squad2-checkpoint.zip": "/content/t5-squad2-checkpoint",
    "/content/drive/MyDrive/T5_Checkpoints/t5-tokenizer-50percent.zip": "/content/t5-tokenizer-50percent"
}

# Ekstrak file ZIP ke folder masing-masing
for zip_file, extract_to in zip_files.items():
    with zipfile.ZipFile(zip_file, 'r') as zip_ref:
        zip_ref.extractall(extract_to)
        print(f"Extracted: {zip_file} -> {extract_to}")


Extracted: /content/drive/MyDrive/T5_Checkpoints/t5-squad2-checkpoint.zip -> /content/t5-squad2-checkpoint
Extracted: /content/drive/MyDrive/T5_Checkpoints/t5-tokenizer-50percent.zip -> /content/t5-tokenizer-50percent


## **Cara mengunakan Pertama(1)**

In [3]:
from transformers import T5ForConditionalGeneration, T5Tokenizer

# Path ke folder yang sudah diekstrak
model_path = "/content/t5-squad2-checkpoint"
tokenizer_path = "/content/t5-tokenizer-50percent"

# Load tokenizer dan model
tokenizer = T5Tokenizer.from_pretrained(tokenizer_path)
model = T5ForConditionalGeneration.from_pretrained(model_path)

print("Model dan tokenizer berhasil dimuat!")


Model dan tokenizer berhasil dimuat!


In [4]:
def answer_question(question, context):
    # Format input sesuai format T5
    input_text = f"question: {question} context: {context}"
    input_ids = tokenizer(input_text, return_tensors="pt").input_ids

    # Generate jawaban
    output_ids = model.generate(input_ids)
    answer = tokenizer.decode(output_ids[0], skip_special_tokens=True)

    return answer

# Contoh tes
context = "Albert Einstein was a theoretical physicist who developed the theory of relativity."
question = "Who developed the theory of relativity?"

print("Pertanyaan:", question)
print("Jawaban:", answer_question(question, context))

Pertanyaan: Who developed the theory of relativity?
Jawaban: Albert Einstein


## **Cara Mengunakan Kedua(2)**

In [7]:
!pip install duckduckgo-search transformers
!pip install deep-translator

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.3/3.3 MB 41.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.3/42.3 kB 2.6 MB/s eta 0:00:00


In [24]:
from duckduckgo_search import DDGS
from deep_translator import GoogleTranslator
from transformers import T5Tokenizer, T5ForConditionalGeneration

# Load model & tokenizer
model_path = "/content/t5-squad2-checkpoint"
tokenizer_path = "/content/t5-tokenizer-50percent"

tokenizer = T5Tokenizer.from_pretrained(tokenizer_path)
model = T5ForConditionalGeneration.from_pretrained(model_path)

def search_context(query):
    """Cari informasi dari DuckDuckGo, ambil beberapa hasil, dan terjemahkan ke bahasa Inggris."""
    context_list = []

    try:
        with DDGS() as ddgs:
            results = ddgs.text(query, max_results=5)  # Ambil 3 hasil pencarian

        for r in results:
            if "body" in r and r["body"]:  # Pastikan ada isi
                context_list.append(r["body"])

        if not context_list:
            return "Context not found."

        # Gabungkan hasil menjadi satu teks
        context = " ".join(context_list)

        # Terjemahkan ke bahasa Inggris jika perlu
        translator = GoogleTranslator(source="auto", target="en")
        translated_context = translator.translate(context)

        return translated_context

    except Exception as e:
        return f"Error: {str(e)}"

def generate_answer(question, context):
    """Jawab pertanyaan dengan model T5"""
    if "Error" in context or "Context not found" in context:
        return "Sorry, I couldn't find relevant information."

    input_text = f"question: {question}  context: {context}"
    input_ids = tokenizer(input_text, return_tensors="pt").input_ids

    # Generate jawaban
    output_ids = model.generate(input_ids, max_length=128)
    return tokenizer.decode(output_ids[0], skip_special_tokens=True)

# Contoh pertanyaan dan pencarian konteks
question = "What is the tallest mountain in the world?"

context = search_context(question)

# Coba jawab pertanyaan
answer = generate_answer(question, context)
print("Translated Context:", context)
print("Jawaban:", answer)


Translated Context: The highest mountains above sea level are generally not the highest mountains above the surrounding terrain, also called the highest free-standing mountains. ... islands are below sea level, and given this consideration Mauna Kea (4,207 m (13,802 ft) above sea level) is the world's tallest mountain and volcano, rising about 10,203 m (33,474 ft) ... The tallest mountain in the world could actually be in the USA, not Nepal. Learn about the different ways to measure the height of mountains and the contenders for the highest mountain in the world. Find out why Mount Everest, Mauna Kea and Chimborazo are all considered the highest in different ways. How do you measure the tallest mountain on Earth? Learn about the different criteria and contenders, from altitude to base to peak to distance from core. The highest peak in India and the third tallest mountain in the world at 28,169 feet, Kanchenjunga welcomes a maximum of 20 to 25 climbers annually—though 2019 saw a record 